Imports

In [ ]:
import pandas as pd
import numpy as np
import datetime

# Define the number of samples for training and testing datasets
n_train = 10000
n_test = 2000

# Define additional fields
location_types = ['Indoor', 'Outdoor']
genders = ['Male', 'Female', 'Other']
hearing_sensitivity = ['Normal', 'Mild', 'Moderate', 'Severe']
health_issues = ['Healthy', 'Diabetes', 'Hypertension', 'Heart Disease', 'Other']
environments = ['Residential', 'Workplace', 'Recreational', 'Traffic', 'Industrial']

# Generate random data
np.random.seed(42)  # For reproducibility

def generate_data(num_samples):
    timestamps = [datetime.datetime.now() - datetime.timedelta(minutes=15*i) for i in range(num_samples)]
    user_ids = np.random.randint(1000, 9999, num_samples)
    noise_levels = np.random.uniform(0, 100, num_samples)  # Mean = 65dB, Std = 10dB
    durations = np.random.normal(60, 20, num_samples)  # Mean = 60 minutes, Std = 20 minutes
    ages = np.random.randint(18, 65, num_samples)
    data = {
        'Timestamp': [ts.strftime('%Y-%m-%d %H:%M:%S') for ts in timestamps],
        'User_ID': user_ids,
        'Location_Type': np.random.choice(location_types, num_samples),
        'Environment': np.random.choice(environments, num_samples),
        'Noise_Level_dB': noise_levels,
        'Duration_Minutes': durations,
        'Age': ages,
        'Gender': np.random.choice(genders, num_samples),
        'Hearing_Protection_Used': np.random.choice(['Yes', 'No'], num_samples),
        'Hearing_Sensitivity': np.random.choice(hearing_sensitivity, num_samples),
        'Health_Issues': np.random.choice(health_issues, num_samples)
    }
    # Simulate hearing damage risk
    data['Hearing_Damage_Risk'] = (data['Noise_Level_dB'] > 95) | (data['Duration_Minutes'] > 480)  # 8 hours threshold
    data['Hearing_Damage_Risk'] = data['Hearing_Damage_Risk'].astype(int)  # Convert to binary
    return pd.DataFrame(data)

# Generate training and testing datasets
train_data = generate_data(n_train)
test_data = generate_data(n_test)

# Save to CSV files
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

print("Data generation complete. Files saved as 'simulated_train_data_extended.csv' and 'simulated_test_data_extended.csv'.")


Data generation complete. Files saved as 'simulated_train_data_extended.csv' and 'simulated_test_data_extended.csv'.


In [ ]:
!pip install joblib

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Drop unnecessary columns
train = train.drop(['Timestamp', 'User_ID'], axis=1)
test = test.drop(['Timestamp', 'User_ID'], axis=1)

data = pd.concat([train, test])


# Handle missing values (if any)
data = data.dropna()

# Define categorical and numerical columns
categorical_columns = ['Location_Type', 'Environment', 'Gender', 'Hearing_Protection_Used', 'Hearing_Sensitivity', 'Health_Issues']
numerical_columns = ['Noise_Level_dB', 'Duration_Minutes', 'Age']

# One-hot encode categorical variables and scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

# Split data into features and target
X = data.drop('Hearing_Damage_Risk', axis=1)
y = data['Hearing_Damage_Risk']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Convert sparse matrices to dense arrays
# X_train = X_train.toarray()
# X_test = X_test.toarray()

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model (optional)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/50
300/300 [==============================] - 3s 6ms/step - loss: 0.1633 - accuracy: 0.9482 - val_loss: 0.1020 - val_accuracy: 0.9488
Epoch 2/50
300/300 [==============================] - 1s 4ms/step - loss: 0.1063 - accuracy: 0.9511 - val_loss: 0.0776 - val_accuracy: 0.9642
Epoch 3/50
300/300 [==============================] - 1s 3ms/step - loss: 0.0903 - accuracy: 0.9576 - val_loss: 0.0648 - val_accuracy: 0.9762
Epoch 4/50
300/300 [==============================] - 1s 3ms/step - loss: 0.0748 - accuracy: 0.9639 - val_loss: 0.0526 - val_accuracy: 0.9846
Epoch 5/50
300/300 [==============================] - 1s 3ms/step - loss: 0.0637 - accuracy: 0.9702 - val_loss: 0.0452 - val_accuracy: 0.9867
Epoch 6/50
300/300 [==============================] - 1s 3ms/step - loss: 0.0539 - accuracy: 0.9756 - val_loss: 0.0403 - val_accuracy: 0.9842
Epoch 7/50
300/300 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 0.9793 - val_loss: 0.0337 - val_accuracy: 0.9896
Epoch 

# New Section

In [ ]:
# Save the entire model to a HDF5 file
model.save("my_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import joblib

# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']

In [ ]:
import tensorflow as tf
import pandas as pd
import joblib

# Load the saved model
loaded_model = tf.keras.models.load_model("my_model.h5")

# Load the preprocessor
loaded_preprocessor = joblib.load('preprocessor.pkl')

sample_data = {
    'Location_Type': 'Indoor',
    'Environment': 'Traffic',
    'Age': '30',
    'Gender': 'Male',
    'Hearing_Protection_Used': 'No',
    'Hearing_Sensitivity': 'Severe',
    'Health_Issues': 'Diabetes',
    'Noise_Level_dB': 76,
    'Duration_Minutes': 6
}

# Convert the sample data to a DataFrame
sample_df = pd.DataFrame([sample_data])

# Preprocess the sample data using the loaded preprocessor
sample_df_processed = loaded_preprocessor.transform(sample_df)

# Make prediction
prediction = loaded_model.predict(sample_df_processed)[0]

# The prediction will be a probability between 0 and 1.
# You can interpret it as the likelihood of hearing damage risk.
print(prediction)


1/1 [==============================] - 0s 64ms/step
[5.537884e-23]


In [ ]:
import pandas as pd
sample_data = {
    'Location_Type': 'Indoor',
    'Environment': 'Traffic',
    'Age': '30',
    'Gender': 'Male',
    'Hearing_Protection_Used': 'Yes',
    'Hearing_Sensitivity': 'Normal',
    'Health_Issues': 'Healthy',
    'Noise_Level_dB': 94,
    'Duration_Minutes': 480
}

# Convert the sample data to a DataFrame
sample_df = pd.DataFrame([sample_data])

# Preprocess the sample data
sample_transformed = preprocessor.transform(sample_df)

# Make a prediction
prediction = model.predict(sample_transformed)[0]

print(prediction[0])


1/1 [==============================] - 0s 25ms/step
2.6889239e-27
